In [1]:
using DataFrames, CSV, JuMP, Gurobi, Random, LibPQ, Statistics

In [2]:
#data = projections for each player
data = DataFrame(CSV.File("dynasty_data.csv"))
#fill missing with 0 in julia
replace!(data.VORP, missing => 0)
data.Name .= data.FirstName .* " " .* data.LastName
data

,DynastySF_ADP,VORP,Team,FirstName,LastName,Position,PlayerID,PositionalRank
,Float64,Float64?,String3,String15,String31,String3,Int64,String7
1,1.1,39.2,KC,Patrick,Mahomes,QB,4046,QB1
2,2.5,35.6,BUF,Josh,Allen,QB,4984,QB2
3,3.1,41.0,PHI,Jalen,Hurts,QB,6904,QB3
4,4.9,42.0,CIN,Joe,Burrow,QB,6770,QB4
5,5.1,45.0,MIN,Justin,Jefferson,WR,6794,WR1
6,6.1,25.1,LAC,Justin,Herbert,QB,6797,QB5
7,7.6,38.6,CIN,Ja'Marr,Chase,WR,7564,WR2
8,8.0,32.9,BAL,Lamar,Jackson,QB,4881,QB6
9,9.0,17.4,JAX,Trevor,Lawrence,QB,7523,QB7


In [3]:
player_to_index = Dict()
#iterate thorugh each row in the data
for i in 1:size(data)[1]
    #get the player name
    player = data[i, :Name]
    #if the player is not in the dictionary, add them
    player_to_index[player] = i
end

In [4]:
qbs = Set()
rbs = Set()
wrs = Set()
tes = Set()

for player in eachrow(data)
    if player.Position == "QB"
        push!(qbs, player.Name)
    elseif player.Position == "RB"
        push!(rbs, player.Name)
    elseif player.Position == "WR"
        push!(wrs, player.Name)
    elseif player.Position == "TE"
        push!(tes, player.Name)
    end
end

In [5]:
function reset_draft()
    drafted = Dict()
    for i in range(1, 10)
        drafted[i] = []
    end
    return drafted
end

reset_draft (generic function with 1 method)

In [6]:
function just_drafted(player, team)
    if length(filter(:Name => n -> n == player, data)[!,1]) < 1
        return "Player not found"
    end
    push!(drafted[team], player)
    return "Team " * string(team) * " drafted " * player * "!"
end

just_drafted (generic function with 1 method)

In [7]:
function view_roster(team)
    return DataFrame(Roster = drafted[team])
end

view_roster (generic function with 1 method)

In [8]:
function players_drafted_opps(team)
    drafted_opps = DataFrame(Name = data.Name, Drafted = fill(false, 321))
    for i in 1:10
        if i == team
            continue
        else
            for player in drafted[i]
                drafted_opps[player_to_index[player], "Drafted"] = true
            end
        end
    end
    return drafted_opps
end

players_drafted_opps (generic function with 1 method)

In [9]:
const GRB_ENV = Gurobi.Env(output_flag=0);

In [10]:
function dynasty_pick(team, temp=[])
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    P = 321 # num_players
    min_qbs = 2
    max_qbs = 3
    min_rbs = 7
    max_rbs = 10
    min_wrs = 7
    max_wrs = 10
    min_tes = 2
    max_tes = 4

    opps = players_drafted_opps(team)

    num_picked = sum(opps[i, "Drafted"] for i in 1:P) + length(drafted[team])
    # for player in temp
    #     opps[player_to_index[player], "Drafted"] = 1
    # end

    picks = [team, 21-team]
    for i in 3:23
        push!(picks, picks[i-2]+20)
    end

    # VARIABLES
    @variable(model, x[i = 1:P], Bin) # whether or not player was drafted by desired team

    # OBJECTIVE FUNCTION
    @objective(model, Max, sum(data[i, "VORP"]*x[i] for i in 1:P))

    # CONSTRAINTS
    # roster must contain already drafted num_players
    @constraint(model, [i = drafted[team]], x[player_to_index[i]] == 1)

    # roster must not contain any opps
    @constraint(model, [i=1:P], x[i]+opps[i,"Drafted"] <= 1)

    # roster must not contain any temps
    @constraint(model, [i=1:length(temp)], x[player_to_index[temp[i]]] == 0)

    # roster must contain 23 players (no kicker/DEF)
    @constraint(model, sum(x[i] for i in 1:P) <= 23)

    # positional constraints
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) >= min_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) <= max_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) >= min_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) <= max_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) >= min_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) <= max_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) >= min_tes)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) <= max_tes)


    # DRAFT POSITION CONSTRAINTS
    #at least 1 pick from set of players after each round
    for pick in 1:23
        @constraint(model, sum(x[i] for i in (picks[pick]-sum(opps[j, "Drafted"] for j in picks[pick]:P)):P) >= 24-pick)
    end

    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)

    roster = DataFrame(Name = String[], Pick = Int64[], ADP = Float64[], VORP = Float64[])
    count = 1
    for i in 1:P
        if value.(x[i]) == 1
            push!(roster, [data[i, "Name"], picks[count], data[i, "DynastySF_ADP"], data[i, "VORP"]])
            count += 1
        end
    end
    println("Total VORP: " * string(round(objective_value(model), digits = 1)))
    return roster
end

dynasty_pick (generic function with 2 methods)

In [11]:
drafted = reset_draft();

In [14]:
just_drafted("Justin Fields", 1)

"Team 1 drafted Justin Fields!"

In [17]:
first_choice = dynasty_pick(8)

Total VORP: 288.6


,Name,Pick,ADP,VORP
,String,Int64,Float64,Float64
1,Ja'Marr Chase,8,7.6,38.6
2,Breece Hall,13,18.5,27.6
3,Cooper Kupp,28,31.5,30.5
4,Davante Adams,33,45.7,23.4
5,Tony Pollard,48,50.3,17.2
6,Jordan Addison,53,55.8,15.6
7,Joe Mixon,68,73.7,13.4
8,Cam Akers,73,84.8,11.9
9,Isiah Pacheco,88,89.5,11.8


In [20]:
second_choice = dynasty_pick(8, [first_choice[1,"Name"]])

Total VORP: 282.4


,Name,Pick,ADP,VORP
,String,Int64,Float64,Float64
1,Bijan Robinson,8,11.4,32.2
2,Breece Hall,13,18.5,27.6
3,Cooper Kupp,28,31.5,30.5
4,Davante Adams,33,45.7,23.4
5,Tony Pollard,48,50.3,17.2
6,Jordan Addison,53,55.8,15.6
7,Joe Mixon,68,73.7,13.4
8,Kenny Pickett,73,81.7,13.5
9,Isiah Pacheco,88,89.5,11.8


In [21]:
third_choice = dynasty_pick(8, [first_choice[1,"Name"], second_choice[1,"Name"]])

Total VORP: 281.6


,Name,Pick,ADP,VORP
,String,Int64,Float64,Float64
1,CeeDee Lamb,8,12.7,31.6
2,Breece Hall,13,18.5,27.6
3,Cooper Kupp,28,31.5,30.5
4,Davante Adams,33,45.7,23.4
5,Tony Pollard,48,50.3,17.2
6,Jordan Addison,53,55.8,15.6
7,Joe Mixon,68,73.7,13.4
8,Cam Akers,73,84.8,11.9
9,Isiah Pacheco,88,89.5,11.8


In [69]:
for player in roster
    if player in tes
        println(player)
    end
end

Tyler Higbee
Zach Ertz


In [ ]:
function dynasty_draft()
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    T = 10 # teams
    R = 25 # roster spots
    P = 500 # num_players -- CHANGE
    min_qbs = 2
    max_qbs = 2
    min_rbs = 4
    max_rbs = 6
    min_wrs = 4
    max_wrs = 6
    min_tes = 2
    max_tes = 3

    # VARIABLES
    @variable(model, x[i = 1:T, j = 1:R] > 0, Int) # R roster spots each for N teams

    # OBJECTIVE FUNCTION - distribution for total expected value - maybe maximize the 25th percentile outcome?
    @objective(model, Max, sum(x[i,j] for i in 1:T, j in 1:R))

    # CONSTRAINTS
    # no player can be on two teams
    @constraint(model, [p = 1:P], sum(x[i,j]==p for i in 1:T, j in 1:R) <= 1)

    # position constraints for each team
    @constraint(model, [i = 1:T], sum(in(x[i,j], qbs) for j in 1:R) >= min_qbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], qbs) for j in 1:R) <= max_qbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], rbs) for j in 1:R) >= min_rbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], rbs) for j in 1:R) <= max_rbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], wrs) for j in 1:R) >= min_wrs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], wrs) for j in 1:R) <= max_wrs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], tes) for j in 1:R) >= min_tes)
    @constraint(model, [i = 1:T], sum(in(x[i,j], tes) for j in 1:R) <= max_tes)

    # already picked constraints
    @constraint(model, [i = 1:T], sum(x[i,j] for j in 1:R) == 16)


    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)


    return value.(x)
end